In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
sequence = input("Paste your DNA sequence:\n")
sequence = sequence.replace(" ", "").replace("\n", "").upper()

print("Sequence length:", len(sequence))

Paste your DNA sequence:
 cttgaacagaaatgactgcgtagaaacatggtctgtatatcataaaccctatatttgaag ttctcttatggacaggcgtgattaacgggtttcagcggtgttactgaaggaatgagatga acgatttaaagcacctgatcggtgtaaccgaccatgactagccgtgttctggtcgtctca aagataatacccgtaggaacggctttgaaagatcatgtctagtggagcgcgaggggccgg gaccatttacaccgtgctggtgccacgatgagtagcattgttaaaatcttcacaatcaca aactgtgactcaggcatacccggaaaacgtgtccataaacatgttcgtcgccaaccggcc tttctgcaagcactgccaaaacaagcgatatgatcttgtgtgttctacagactatgttca ggtcccccgctttggaatcctagcgaacgtctagatggtcacagccggtactgcattctc gctcacgtaagtgcaatgtgggtaagtatttctccacagaacgaatgggggaatgtcgag ctgtcaacaggtacttgccctgtgtgcacaactgtgggacagtcactaccccggccgagt agtgctgcactcaatctctaataaccatacctctaattaaggtgattgcccaagtccgaa tcacacagatgcaggtacggttgttcatcggtcttgcatgctgagttttaatgggcccgc tgaatacgttcaaacgagctgcggaccttgccggttcacctcccgctgattgaaatgggg tatgcaactcactagtatctgtttagaaatatcagaagcatagacaaatccgtaaattta tgttgggatatgatttaggtctgttactcgcggtgacgacctcactaggccatcacgacg tggtttgctgctaagactcggggctaggaccatatgtgatggctgagagttggcctgat

Sequence length: 1000


In [20]:

def find_pam_sites(seq):
    sites = [] #Starts an empty list where each pam guide will be stored

    for i in range(len(seq) - 2):
        if seq[i+1:i+3] == "GG": 
            pam = seq[i:i+3] #Stores the pam site
            guide_start = i - 20 # Takes the 20 bp upstream of pam site as gRNA
            guide_end = i

            if guide_start >= 0: #Skips PAM sites that are too close to the 5' end
                guide = seq[guide_start:guide_end]
                sites.append({ 
                    "pam_index": i,
                    "pam": pam,
                    "guide": guide #Adds where the PAM sequence starts, the 3bp in the PAM, and the 20bp upstream to the sites dictionary
                })
    return pd.DataFrame(sites) #Creates a Data Frame of the sites dictionary with all the PAM sites
    
df = find_pam_sites(sequence)

def gc_content(seq):
    return (seq.count("G") + seq.count("C")) / len(seq)# Adds all the Gs and Cs together and divides by the total number of Bases to get a fraction

df["gc"] = df["guide"].apply(gc_content) # For each row in the dataframe, it takes all of the gs and cs and creates a new column in the table
center = len(sequence) / 2

df["center_distance"] = abs(df["pam_index"] - center) #Takes distance from center,
# Just for ease of cloning, better when PAM is closer to center

df["pam_score"] = (
    df["gc"] * 0.6 +
    (1 / (df["center_distance"] + 1)) * 0.4 #Takes weight to gc content more than center distance
) #Added 1 to each value in case center distance is 0 and we dont want to divide by 0

df_sorted = df.sort_values("pam_score", ascending=False)

df_sorted.head() # Sorts the scores anc creates a top 5 scores
# This isnt the final score, I still need to take a few more factors into play
def has_homopolymer(seq, length=4):
    # Check if there is any run of the same base of length "length"
    return any(base * length in seq for base in "ATCG")

df["has_homopolymer"] = df["guide"].apply(has_homopolymer)

# Numeric penalty if true: -1 if bad, 0 if fine
df["homopolymer_penalty"] = df["has_homopolymer"].apply(
    lambda present: -1 if present else 0
)

def seed_gc(seq):
    # Take positions 3–8 (Python indices 2:8) GC content in these 6 bases matters a lot more
    seed = seq[2:8]
    return (seed.count("G") + seed.count("C")) / len(seed)

df["seed_gc"] = df["guide"].apply(seed_gc)

def pam_prox_gc(seq):
    # Uses last 4 bases of the guide
    proximal = seq[-4:]
    return (proximal.count("G") + proximal.count("C")) / len(proximal)

df["pam_prox_gc"] = df["guide"].apply(pam_prox_gc)

# Turn GC fraction into a score where 0.5 is best
def gc_optimal_score(gc):
    # 1 at gc = 0.5, 0 at gc = 0 or 1
    return 1 - 4 * (gc - 0.5)**2

# Apply this to overall GC, seed GC, and PAM-proximal GC
df["gc_score"] = df["gc"].apply(gc_optimal_score)
df["seed_gc_score"] = df["seed_gc"].apply(gc_optimal_score)
df["pam_prox_gc_score"] = df["pam_prox_gc"].apply(gc_optimal_score)

df["final_score"] = (
    df["gc_score"] * 0.3
    + df["seed_gc_score"] * 0.2
    + df["pam_prox_gc_score"] * 0.2
    + (1 / (df["center_distance"] + 1)) * 0.2
    + df["homopolymer_penalty"] * 0.1
)

In [21]:
df_sorted = df.sort_values("final_score", ascending=False)
df_sorted.head()


,pam_index,pam,guide,gc,center_distance,pam_score,has_homopolymer,homopolymer_penalty,seed_gc,pam_prox_gc,gc_score,seed_gc_score,pam_prox_gc_score,final_score
26,499,GGG,CGCTCACGTAAGTGCAATGT,0.50,1.0,0.500000,False,0,0.666667,0.25,1.00,0.888889,0.75,0.727778
30,528,GGG,TTTCTCCACAGAACGAATGG,0.45,28.0,0.283793,False,0,0.500000,0.50,0.99,1.000000,1.00,0.703897
36,672,AGG,AGTCCGAATCACACAGATGC,0.50,172.0,0.302312,False,0,0.500000,0.50,1.00,1.000000,1.00,0.701156
19,320,CGG,AACTGTGACTCAGGCATACC,0.50,180.0,0.302210,False,0,0.500000,0.50,1.00,1.000000,1.00,0.701105
53,919,GGG,GTGGTTTGCTGCTAAGACTC,0.50,419.0,0.300952,False,0,0.500000,0.50,1.00,1.000000,1.00,0.700476


In [22]:
df_sorted.tail()

,pam_index,pam,guide,gc,center_distance,pam_score,has_homopolymer,homopolymer_penalty,seed_gc,pam_prox_gc,gc_score,seed_gc_score,pam_prox_gc_score,final_score
40,712,GGG,CTTGCATGCTGAGTTTTAAT,0.35,212.0,0.211878,True,-1,0.500000,0.0,0.91,1.000000,0.0,0.373939
15,237,CGG,TCTAGTGGAGCGCGAGGGGC,0.70,263.0,0.421515,True,-1,0.500000,1.0,0.84,1.000000,0.0,0.352758
39,711,TGG,TCTTGCATGCTGAGTTTTAA,0.35,211.0,0.211887,True,-1,0.333333,0.0,0.91,0.888889,0.0,0.351721
48,856,AGG,TTTATGTTGGGATATGATTT,0.25,356.0,0.151120,False,0,0.166667,0.0,0.75,0.555556,0.0,0.336671
16,238,GGG,CTAGTGGAGCGCGAGGGGCC,0.75,262.0,0.451521,True,-1,0.500000,1.0,0.75,1.000000,0.0,0.325760
